In [1]:
from fileToolbox import *

In [2]:
# przejść po wszystkich osobach i filmach, które ta osoba obejrzała
# dla każdej osoby i k-tego filmu, który obejrzała - przejść po wszystkich pozostałych osobach i sprawdzić, które wystawiły taką samą ocenę
# może zrobić słowniki, które będą zliczały punkty dla każdej z osób {person_id_1: punkty, person_id_2: punkty...} UWAGA: dla każdej osoby osobny słownik - 200 słowników
# jak osoba dała taką samą ocene to +2pkt, jak różniącą się o 1 to + 1pkt, w innych przypadkach 0
# i weźmiemy np. średnią ocenę z o-osób, które miały najwięcej punktów dla danej osoby

In [3]:
set_of_person_ids = set(train_df["person_id"])
print(len(set_of_person_ids))

358


In [16]:
# tworzymy słownik dla każdej osoby z person_id i wewnętrznym słownikiem {person_id_1: punkty, person_id_2: punkty...}
json_list = [] 

for person in set_of_person_ids:
    person_dict = {}
    for p in set_of_person_ids:
        if p==person:
            continue
        person_dict[p] = 0 # inicjalizujemy punkty na 0
    json_list.append({'person_id': person, "points" : person_dict})

In [17]:
counter = 0
for person in json_list:
    person_df = filterDataFrame("person_id", person['person_id'], train_df)
    person_movies = getPersonMovies(person_df)
    # dla każdego filmu, trzeba przejść po wszytskich osobach i zliczyć punkty
    for movie in person_movies:
        for p in person['points'].keys():
            if movie in getPersonMovies(filterDataFrame("person_id", p, train_df)):
                rate_main_person = int(filterDataFrame("movie_id", movie, person_df)['rate'])
                rate_comparable_person = int(filterDataFrame("movie_id", movie, filterDataFrame("person_id", p, train_df))['rate'])
                if rate_main_person == rate_comparable_person:
                    points = 2
                elif (rate_main_person-rate_comparable_person==1) or (rate_comparable_person-rate_main_person==1):
                    points = 1
                else:
                    points = 0
                person['points'][p] += points
#                 print("Movie: " + str(movie) + "Rate person: " + str(person['person_id']) + ' ' + str(rate_main_person) + ', rate comparable: ' + str(p) + ' ' + str(rate_comparable_person))
    

In [18]:
# Convert the list to a JSON-formatted string
json_data = json.dumps(json_list, indent=2)

# Save the JSON data to a file
with open('json/person_similarity.json', 'w') as file:
    json.dump(json_list, file, indent=2)

In [21]:
with open('json/person_similarity.json', 'r', encoding='utf-8') as file:
    PERSON_SIMILARITY = json.load(file)

In [110]:
# k - parametr określający na ile najbardziej podobnych osób chcemy patrzeć
def getMostSimilarPersonIDs(person_id, k):
    for person in PERSON_SIMILARITY:
        if person["person_id"]==person_id:
            points = person['points']
            sorted_points = dict(sorted(points.items(), key=lambda item: item[1], reverse=True))
            first_k_elements = dict(list(sorted_points.items())[:k])
    return list(first_k_elements.keys())

In [44]:
tmp = getMostSimilarPersonIDs(5,5)

In [95]:
def getMeanFromSimilarPersonIDs(list_of_similar_person_ids, movie_id, df):
    res = 0
    rated = 0 # licznik do średniej, bo niektórych osób może nie być i by było +0 i by zakłamało wynik przy średniej
    for person_id in list_of_similar_person_ids:
        movies = filterDataFrame('person_id', int(person_id), df)
        if movie_id in list(movies['movie_id']):
            res += int(filterDataFrame('movie_id', movie_id, movies)['rate'])
            rated += 1
    return int(res / rated) # średnia z wyników

In [111]:
def similarityClassifier(person_id, movie_id, k, df):
    flag = True
    while(flag):
        list_of_similar_person_ids = getMostSimilarPersonIDs(person_id, k)
        res = 0
        rated = 0 # licznik do średniej, bo niektórych osób może nie być i by było +0 i by zakłamało wynik przy średniej
        for person in list_of_similar_person_ids:
            movies = filterDataFrame('person_id', int(person), df)
            if movie_id in list(movies['movie_id']):
                res += int(filterDataFrame('movie_id', movie_id, movies)['rate'])
                rated += 1
        if rated != 0: # jeśli znaleźliśmy chociaż jednego similara to koniec
            flag=False
        else:
            k += 3 # zwiększamy zakres poszukiwań
    return int(res / rated)

In [79]:
train_list = []
for index, row in train_df.iterrows():
#     print(index)
#     train_list.append({'person_id' : int(row['person_id']), 'movie_id' : int(row['movie_id']), 'rate' : int(row['rate'])})
    train_list.append([int(row['person_id']), int(row['movie_id']), int(row['rate'])])

In [80]:
train_list[:5]

[[1642, 137, 4], [1642, 1, 5], [1642, 136, 5], [1642, 2, 3], [1642, 139, 5]]

In [88]:
length = int(len(train_list)*0.2)

In [112]:
test = train_list[:length]

In [113]:
test_x = [elem[:2] for elem in test]
test_y = [elem[2] for elem in test]

In [ ]:
k = 5
results = []
for index, elem in enumerate(test_x):
    print(index)
    results.append(similarityClassifier(elem[0], elem[1], k, train_df))

In [115]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_y, results)

In [118]:
round(accuracy, 2)

0.52

In [ ]:
k = 10
results = []
for index, elem in enumerate(test_x):
    results.append(similarityClassifier(elem[0], elem[1], k, train_df))

In [121]:
accuracy_score(test_y, results)

0.43202979515828677

In [122]:
k = 1
results = []
for index, elem in enumerate(test_x):
    results.append(similarityClassifier(elem[0], elem[1], k, train_df))

In [123]:
accuracy_score(test_y, results)

0.5921787709497207

Czyli najlepiej brać statystycznie 1 similara do określania wyniku

In [124]:
COUNTER = 0
def applySimilarityClassifier(row):
    global COUNTER
    print(COUNTER)
    COUNTER += 1
    movie_id = row['movie_id']
    person_id = row['person_id']    
    rate = similarityClassifier(person_id, movie_id, 1, train_df)
    return rate

In [ ]:
# Apply the function to update the "rate" column
task_df['rate'] = task_df.apply(applySimilarityClassifier, axis=1)

In [126]:
task_df.to_csv('submission_similarity.csv', index=False)